# 0. Import needed Libraries

In [ ]:
# the 3 main libraries
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
# librairies to do nicer plots...
import cartopy.crs as ccrs     # only for plots with projections and continents maps
#import hvplot.xarray           # only for "fancy" plots

# 1. xarray basic manipulation:

 - Open file

In [ ]:
data = xr.open_dataset('sst.mnmean.nc')

 - explore **[Dataset](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.html)**

In [ ]:
data  # file Dataset

In [ ]:
data.coords

In [ ]:
data.time.size

In [ ]:
data.data_vars

 - explore **[DataArray](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.html)**

In [ ]:
data.sst  # sst DataArray http://xarray.pydata.org/en/stable/generated/xarray.DataArray.html

In [ ]:
print(data.sst.shape)
print(data.sst.ndim)
print(data.sst.dims)

data index selection: **[isel()](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.isel.html)**  

In [ ]:
data.sst.isel(lon=100, lat=50, time=0)     # select the point defined by its indexes (i=100, j=50, t=0)

data value selection: **[sel()](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.sel.html)**  

In [ ]:
data.sst.sel(lon=180.5, lat=0.5)   # time series for a point defined by its location (lon = 180.5, lat = 0.5)
                                   # --> Warning: lon=180.5 and lat=0.5 must exist 
data.sst.sel(lon=180.0, lat=0.0, method='nearest')   # time series for a point defined near its location

the **[slice](https://docs.python.org/3.8/library/functions.html#slice)** object  

In [ ]:
data.sst.sel(lon=180.5, lat=slice(1.0,-1.0))  # Warning : respect latitude order 1.0 before -1.0

different syntaxes accepted for the time dimension

In [ ]:
print( data.sst.isel(time=200).shape )
print( data.sst.sel(time='2000-01-01T00:00:00.000000000').shape )
print( data.sst.sel(time='2000-01-01').shape )
print( data.sst.sel(time='2000-01').shape )
print( data.sst.sel(time='2000').shape )
print( data.sst.sel(time=slice('2000','2002')).shape )

combine isel and sel selections

In [ ]:
data.sst.isel(time=200).sel(lon=180.5, lat=slice(1.0,-1.0))

the **[GroupBy](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.groupby.html)** object  

In [ ]:
data.sst.groupby('time.month').mean(dim='time').shape

# 2. reduce memory footprint

In [ ]:
data = data.sel(time=slice('1982','2013'))   # keep only "complete" years
data = data.sel(lat=slice(60, -60))      # get rid of data out of 60S-60N

# 3. a few basic plots
See xarray doc on [visualization](https://tutorial.xarray.dev/fundamentals/04.0_plotting.html)

SST time series on 2 different SST time series on 2 different points, [pyplot options](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html)

In [ ]:
fig, axes = plt.subplots(2,1,sharex=True)
data.sst.isel(lon=100, lat=50).plot(ax=axes[0])
data.sst.sel(lon=180.5, lat=0.5).plot(ax=axes[1])
plt.show()

Spatial average of monthly mean 

In [ ]:
data.sst.groupby('time.month').mean(dim='time').mean(dim=('lon','lat')).plot()

Get the land-sea mask

In [ ]:
data_mask = xr.open_dataset('lsmask.nc')
data_mask

In [ ]:
mask = data_mask.mask                     # get the DataArray from the Dataset
mask = mask.sel(lat=slice(60.0, -60.0))   # select mask.mask and reduce memory footprint: get rid of data out of 60S-60N
mask = mask.squeeze()                     # get rid of the time dimension in the variable mask.mask
mask.plot()

Simple SST maps without and with masked values.
[colorbars list](https://matplotlib.org/tutorials/colors/colormaps.html)

In [ ]:
fig, axes = plt.subplots(2,1)
sst2d = data.sst.isel(time=100)
sst2d.plot(ax=axes[0])
sst2d = sst2d.where(mask == 1.)
sst2d.plot(vmin=-2, vmax=32, cmap='gist_rainbow_r',ax=axes[1])
plt.show()

Map with continents and [projection](https://scitools.org.uk/cartopy/docs/latest/crs/projections.html) 

In [ ]:
p = data.sst.isel(time=200).where(mask == 1.).plot(
    subplot_kws=dict( projection=ccrs.PlateCarree(central_longitude=200), facecolor='gray'),
    transform=ccrs.PlateCarree(),vmin=-2, vmax=32, cmap='gist_rainbow_r'
)
p.axes.set_global()
p.axes.coastlines()
p.axes.gridlines()

Widget maps with continents and projection

In [ ]:
#proj = ccrs.PlateCarree(central_longitude=200)
#proj = ccrs.LambertCylindrical(central_longitude=200)

#data.sst.isel(time=slice(1, 13)).where(mask.mask == 1.).hvplot.quadmesh(
#    'lon', 'lat', clim=(-2,32), projection=proj, project=True, global_extent=True, 
#    cmap='gist_rainbow_r', rasterize=True, dynamic=False, coastline=True, 
#    frame_width=500, widget_location='bottom', widget_type='scrubber')

Longitude-time plot around the equator "[hovmoller diagram](https://fr.wikipedia.org/wiki/Diagramme_de_Hovm%C3%B6ller)"

In [ ]:
data.sst.sel(lat=slice(1.5,-1.5)).mean(dim='lat').plot()

# 4. basic computation:

Compute spatial weight:  
Earth is not flat! Regular lat-lon grid cells area varies with the cosinus of the latitude (see [sperical coordinates](https://mathinsight.org/spherical_coordinates)).  
  -> weight data with mask * cos( lat )

In [ ]:
weights = np.cos( np.deg2rad(mask.lat) )
weights = mask*weights
weights.plot()

Visualize the weights

In [ ]:
data.sst.weighted(weights).mean(("time", "lon")).plot()

Comparison of horizontal mean: masked and [weighted](http://xarray.pydata.org/en/stable/computation.html?highlight=rolling#weighted-array-reductions)

In [ ]:
data.sst.mean(("lon", "lat")).plot(label="unweighted")
weights = np.cos(np.deg2rad(data.lat))
data.sst.weighted(weights).mean(("lon", "lat")).plot(label="weighted")
weights = np.cos(np.deg2rad(mask.lat))
weights = mask*weights
data.sst.weighted(weights).mean(("lon", "lat")).plot(label="weighted+mask")
plt.legend()

Mean seasonal cycle

In [ ]:
a = data.sst.groupby('time.month').mean(dim='time')
#a.mean(dim=('lon','lat')).plot(label="unweighted")
fig, axes = plt.subplots(3,1, figsize=(10, 10), constrained_layout=True)
a.weighted(weights).mean(dim=('lon','lat')).plot(label="global mean", ax=axes[0])
axes[0].set_title('global mean')
a.sel(lon=150.5, lat=2.5).plot(ax=axes[1])
axes[1].set_title('West Eq. Pac.')
a.sel(lon=300.5, lat=45.5).plot(ax=axes[2])
axes[2].set_title('North-West Atl.')

[Rolling](http://xarray.pydata.org/en/stable/computation.html?highlight=rolling#rolling-window-operations) mean

In [ ]:
a = data.sst.weighted(weights).mean(("lon", "lat"))
a.plot(label="weighted+mask")
b = a.rolling(time=12, center=True).mean()
b.plot(label="rolling 1 year")
b = a.rolling(time=12*5, center=True).mean()
b.plot(label="rolling 5 years")
plt.legend()
c = a.rolling(time=12*10, center=True).mean()
c.plot(label="rolling 10 years")
plt.legend()

Global yearly mean

In [ ]:
a = data.sst.groupby('time.year').mean(dim='time')   # yearly mean
a.weighted(weights).mean(dim=('lon','lat')).plot(label="weighted+mask")
#a.mean(dim=('lon','lat')).plot(label="unweighted")
plt.legend()

Total standard deviation : [std](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.std.html?highlight=std) method 

In [ ]:
a = data.sst.std(dim='time').where(mask == 1.)
fig, axes = plt.subplots(2,1)
a.plot(vmin=0,vmax=6,ax=axes[0])
a.sel(lon=slice(260.5,320.5),lat=slice(50.5,10.5)).plot(vmin=0,vmax=6,ax=axes[1])

Seasonal standard deviation

In [ ]:
a = data.sst.groupby('time.month').mean(dim='time')
#(a.max(dim='month') - a.min(dim='month')).where(mask.mask == 1.).plot(vmin=0,vmax=12)
a.std(dim='month').where(mask == 1.).plot(vmin=0,vmax=6)

# 5. linear trend:

Long term mean trend ([linear regression](http://xarray.pydata.org/en/stable/computation.html?highlight=polyval#fitting-polynomials) with [polyfit](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.polyfit.html?highlight=polyfit) and [polyval](http://xarray.pydata.org/en/stable/generated/xarray.polyval.html?highlight=polyval) methonds)

In [ ]:
linfit = data.sst.polyfit('time', 1)               # compute linear regression coefficients
linfit.polyfit_coefficients

In [ ]:
slope = linfit.polyfit_coefficients.isel(degree=0)   # Polynomial coefficients, highest power first
ns_century = 1.e9*3600.*24.*365.*100.                # scaling: nono-seconds to century
(slope*ns_century).where(mask == 1.).plot(vmin=-5,vmax=5,cmap='RdBu_r')
plt.title('Trend Slope in deg/century')

Detrend SST

In [ ]:
trend = xr.polyval(coord=data.time, coeffs=linfit.polyfit_coefficients)   # SST trend
sst_detrend = data.sst - trend.values + data.sst.mean(dim='time')         # detrended SST

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(6,3), constrained_layout=True)
a = data.sst.groupby('time.year').mean(dim='time')      # yearly mean
a.weighted(weights).mean(dim=('lon','lat')).plot(ax=axes[0], label="org")
a = trend.groupby('time.year').mean(dim='time')         # yearly mean
a.weighted(weights).mean(dim=('lon','lat')).plot(ax=axes[0], label="trend")
axes[0].legend()
axes[0].set_title('With trend')
#
a = sst_detrend.groupby('time.year').mean(dim='time')   # yearly mean
a.weighted(weights).mean(dim=('lon','lat')).plot(ax=axes[1], label="detrend")
axes[1].legend()
axes[1].set_title('Without trend')

Detrended interannual anomaly

In [ ]:
sstbymth = sst_detrend.groupby("time.month")
mthclim = sstbymth.mean("time")   # detrended climatological months (seasonal cycle)
sstanom = sstbymth - mthclim      # detrended interannual annomaly

interannual anomaly (with/without trend)

In [ ]:
a = data.sst.weighted(weights).mean(dim=('lon','lat'))
clim = a.groupby("time.month").mean("time")   # climatological months (seasonal cycle)
anom = a.groupby("time.month") - clim         # interannual annomaly
anom.plot(label="org")
#
sstanom.weighted(weights).mean(dim=('lon','lat')).plot(label="detrended")
plt.legend()